In [ ]:
using Pkg
Pkg.activate("..")
using Revise
using IMAS;

In [3]:
using IMAS


#= ============== =#
# initialization #
#= ============== =#

PP = IMAS.imas_parameters[:PLASMA_PARAMETERS]
PM = IMAS.imas_parameters[:PHYSICS_MODELS]

core1D = IMAS.core_profiles__profiles_1d()

n = 11

core1D.grid.rho_tor_norm = range(0.0, 1.0, length=n)
core1D.electrons.density = (rho_tor_norm;_...) -> PP[:ne0] .* (1.0 .- rho_tor_norm.^2).^PP[:Sn]
core1D.electrons.temperature = (rho_tor_norm;_...) -> PP[:Te0] .* (1.0 .- rho_tor_norm.^2).^PP[:St]

core1D.j_total = (x;_...) -> (1.0 .- x.^2).^PP[:Sj]

equil = IMAS.equilibrium__time_slice()
equil.profiles_1d.psi = range(0.0, 1.0, length=n)
equil.profiles_1d.elongation = (psi;_...) -> psi .* 0.0 .+ PP[:elongation]
equil.profiles_1d.geometric_axis.r = (psi;_...) -> psi .* 0.0 .+ PP[:R0]
print(equil.profiles_1d.geometric_axis.z)

bootstrapCoefficient = IMAS.collisionless_bootstrap(PM[:bootstrapModel], PP[:elongation], PP[:St], PP[:Sn], PP[:Sj], PP[:Zeff])
println(bootstrapCoefficient)

# maxStableElongation(aspectRatio) = 2.43 + 65.0 * exp(-aspectRatio / 0.376)

# #elongation_fraction = elongation / maxStableElongation

In [2]:
core_profiles = IMAS.core_profiles()
resize!(core_profiles.profiles_1d, 1)
profiles_1d = core_profiles.profiles_1d[1]

n = 5
ne0 = 1e19
Te0 = 1e3

profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=n)
profiles_1d.electrons.density = ne0 .* (1.0 .- profiles_1d.grid.rho_tor_norm .^ 2)  # numerical array
profiles_1d.electrons.temperature = (rho_tor_norm;_...) -> Te0 .* (1.0 .- rho_tor_norm .^ 2) # user defined function

#profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=11)
#println(profiles_1d.electrons.density)

println(profiles_1d.electrons.pressure) # expression from transformation library
profiles_1d.electrons.temperature = profiles_1d.electrons.temperature .* 2
println(profiles_1d.electrons.pressure) # expression from transformation library


[1602.18, 1408.166015625, 901.2262499999999, 306.667265625, 0.0]
[3204.36, 2816.33203125, 1802.4524999999999, 613.33453125, 0.0]


In [3]:
profiles_1d.electrons.density = 2.0 .* profiles_1d.electrons.density
println(profiles_1d.electrons.pressure) # expression is updated based on new density value --> consistency

[6408.72, 5632.6640625, 3604.9049999999997, 1226.6690625, 0.0]


In [4]:
profiles_1d.electrons.temperature = 2.0 .* profiles_1d.electrons.temperature
println(profiles_1d.electrons.pressure) # expression keeps being updated based on new density value --> consistency

[12817.44, 11265.328125, 7209.8099999999995, 2453.338125, 0.0]


In [5]:
filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
data  = IMAS.json2imas(filename; verbose=false)
time_slice = data.equilibrium.time_slice[1]
pres0=time_slice.profiles_1d.pressure();

LoadError: MethodError: objects of type Vector{Float64} are not callable
Use square brackets [] for indexing an Array.

In [39]:


function test_gradient(mul)
    a = pres0 .* mul
    print(typeof(a))
    time_slice.profiles_1d.pressure = a
    #time_slice.global_quantities.energy_mhd
#    time_slice.global_quantities.energy_mhd
    sum(time_slice.profiles_1d.pressure)
    sum(a)
end

#test_gradient(2)

Zygote.gradient(test_gradient, 1)

In [ ]:
profiles_1d = IMAS.equilibrium__time_slice___profiles_1d()

Pkg.add("Zygote")
using Zygote

profiles_1d.psi = range(0.0, 1.0, length=5)

function test_gradient(mul)
    a = profiles_1d.psi .* mul
    profiles_1d.psi = a * mul
#    sum(a)
    sum(profiles_1d.psi)
end

# #test_gradient(2)

Zygote.gradient(test_gradient, 1)